In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from lightfm import LightFM, data, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
%matplotlib inline

### Считывание данных рейтингов и описаний книг

In [2]:
ratings = pd.read_csv("C:/Users/v_rusakevich/Documents/аметист/recom/BX-Book-Ratings.csv", sep=';', encoding = "ISO-8859-1")
ratings.rename(columns = {'User-ID':'user_id', 'ISBN': 'item_id', 'Book-Rating':'weight'}, inplace=True)
ratings.head(25)

,user_id,item_id,weight
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
6,276736,3257224281,8
7,276737,0600570967,6
8,276744,038550120X,7
9,276745,342310538,10


In [3]:
books = pd.read_csv("C:/Users/v_rusakevich/Documents/аметист/recom/BX-Books.csv", sep=';', encoding = "ISO-8859-1", error_bad_lines=False)
books.head(25)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...


### Формирование датасета для обучения

In [4]:
# Создание экземпляра класса датасета и передача в него пользователей и элементов
dataset = data.Dataset()
dataset.fit((row.user_id for row in ratings.itertuples()),
            (row.item_id for row in ratings.itertuples()))

In [5]:
# Создание матриц пользователей-элементов и весов
(matrix, weights) = dataset.build_interactions((row.user_id, row.item_id, row.weight) for row in ratings.itertuples())

In [6]:
# Разбиение матриц на тренировочные и тестовые
(train, test) = cross_validation.random_train_test_split(matrix, random_state=np.random.RandomState(seed=1))
(train_weight, test_weight) = cross_validation.random_train_test_split(weights, random_state=np.random.RandomState(seed=1))

###### Обучение первой модели на функции потерь logistic:

In [7]:
model1 = LightFM(no_components=30)
model1.fit(train, sample_weight=train_weight, epochs=20)

In [8]:
# roc-auc метрика на тренировочной выборке
train_auc = auc_score(model1, train).mean()
train_auc

0.8736942

In [9]:
# roc-auc метрика на тестовой выборке
test_auc = auc_score(model1, test).mean()
test_auc

0.7344953

###### Обучение второй модели на функции потерь Bayesian Personalised Ranking:

In [10]:
model2 = LightFM(no_components=30, loss='bpr')
model2.fit(train, sample_weight=train_weight, epochs=20)

In [11]:
# roc-auc метрика на тренировочной выборке
train_auc = auc_score(model2, train).mean()
train_auc

0.7802796

In [12]:
# roc-auc метрика на тестовой выборке
test_auc = auc_score(model2, test).mean()
test_auc

0.62315875

###### Обучение третьей модели на функции потерь Weighted Approximate-Rank:

In [13]:
model3 = LightFM(no_components=30, loss='warp')
model3.fit(train, sample_weight=train_weight, epochs=20)

In [14]:
# roc-auc метрика на тренировочной выборке
train_auc = auc_score(model3, train).mean()
train_auc

0.8809248

In [15]:
# roc-auc метрика на тестовой выборке
test_auc = auc_score(model3, test).mean()
test_auc

0.72153366

Лучше всего качество получилось у функций потерь logistic и warp - это первая и третья модель

###### Рекомендации на первой и третьей моделях:

In [16]:
def display_recommended_items(model, user_id, data):
    '''Функция вывода на печать известных рейтингов у пользователя и рекомендации для него'''
    scores = model.predict(user_id, np.arange(data.shape[1]))
    scores = np.argsort(-scores)[0:10]
    items = {v:k for k, v in dataset.mapping()[2].items()}
    users = {v:k for k, v in dataset.mapping()[0].items()}

    known_positives = ratings[ratings['user_id']==users[user_id]].sort_values('weight', ascending=False)
    print('Known positives:')
    for i in known_positives.itertuples():
        author = books[books['ISBN'] == i.item_id]['Book-Author'].values
        title = books[books['ISBN'] == i.item_id]['Book-Title'].values
        if author != '' or title != '':
            print('{} - {} - {}'.format(*author, *title, i.weight))
    print('############################################################')
    print('Recommendations:')
    for i in scores:
        author = books[books['ISBN'] == items[i]]['Book-Author'].values
        title = books[books['ISBN'] == items[i]]['Book-Title'].values
        if author != '' or title != '':
            print('{} - {}'.format(*author, *title))

In [17]:
display_recommended_items(model1, 388, matrix)

Herman Melville - Billy Budd and Other Tales (Signet Classic) - 7
############################################################
Alice Sebold - The Lovely Bones: A Novel
Dan Brown - The Da Vinci Code
Rich Shapero - Wild Animus
Anita Diamant - The Red Tent (Bestselling Backlist)
Rebecca Wells - Divine Secrets of the Ya-Ya Sisterhood: A Novel
John Grisham - A Painted House
Tracy Chevalier - Girl with a Pearl Earring
J. K. Rowling - Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Sue Monk Kidd - The Secret Life of Bees


In [18]:
display_recommended_items(model3, 388, matrix)

Herman Melville - Billy Budd and Other Tales (Signet Classic) - 7
############################################################
Rich Shapero - Wild Animus
Eric Schlosser - Fast Food Nation: The Dark Side of the All-American Meal
David P. Jordan - Out Of The Night (Harlequin Presents, No. 1427)
Reiji Saiga - Real Bout High School, Book 4
Herman Melville - Billy Budd and Other Tales (Signet Classic)
Helen Fielding - Bridget Jones's Diary
Marc Alexander - Not After Nightfall
Dawn Reno - All That Glitters
Bryan E. Robinson - The Feel Good Book: 1001 Ways to Be Happy
Ingrid Seward - The Queen &amp; Di : The Untold Story


Рекомендации у двух моделей разные, хотя одно совпадение есть - Rich Shapero - Wild Animus